In [549]:
import json
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd
import re
import seaborn as sns

In [550]:
with open('../data/flutter/flutter_issues_labeled.json') as json_data:
    issues = json.load(json_data)
print("Number of issues: " + str(len(issues)))

Number of issues: 7170


In [551]:
with open('../data/flutter/flutter_issues_prs_final.json') as json_data:
    issues_prs = json.load(json_data)
print("Number of issues: " + str(len(issues_prs)))

success = []
failed = []
partial = []
for issue in issues_prs:
    if issue['merged_prs'] or issue['master_commits']:
        success.append(issue)
    elif issue['local_commits'] or issue['failed_prs']:
        #print(issue)
        partial.append(issue)
    else:
        failed.append(issue)
print("Number of successful issues: " + str(len(success)))
print("Number of partially successful issues: " + str(len(partial)))
print("Number of failed issues: " + str(len(failed)))

Number of issues: 11148
Number of successful issues: 3551
Number of partially successful issues: 708
Number of failed issues: 6889


In [552]:
#TODO: fix the extra PRs from code #s

In [553]:
issue_map = {}
for issue in issues_prs:
    completed_by = set()
    failed_by = set()
    for pr in issue['merged_prs']:
        if pr['author']:
            completed_by.add(pr['author'])
    for commit in issue['master_commits']:
        if commit['author']:
            completed_by.add(commit['author'])
    for pr in issue['failed_prs']:
        if pr['author'] and pr['author'] not in completed_by:
            failed_by.add(pr['author'])
    for commit in issue['local_commits']:
        if commit['author'] and commit['author'] not in completed_by:
            failed_by.add(commit['author'])
    issue_map[issue['issue_num']] = {}
    issue_map[issue['issue_num']]['completed_by'] = completed_by
    issue_map[issue['issue_num']]['failed_by'] = failed_by

In [554]:
# create a data frame from the list of issues
df_list = []
for issue in labeled_issues:
        if issue['number'] not in issue_map:
            continue
        df_dict = {}
        df_dict['comments'] = issue['comments']
        if (not issue['body']):
            issue['body'] = ""
        df_dict['title'] = issue['title']
        df_dict['issue_num'] = issue['number']
        df_dict['body'] = issue['body']
        df_dict['closed_date'] = pd.to_datetime(issue['closed_at'])
        df_dict['created_date'] = pd.to_datetime(issue['created_at'])
        df_dict['completed_by'] = issue_map[issue['number']]['completed_by']
        df_dict['failed_by'] = issue_map[issue['number']]['failed_by']
        if len(df_dict['completed_by']) == 0 and len(df_dict['failed_by']) == 0:
            continue
        df_dict['labels'] = [label['name'] for label in issue['labels']]
        df_dict['assignees'] = [assignee['login'] for assignee in issue['assignees']]
        for assignee in df_dict['assignees']:
            if assignee not in df_dict['completed_by'] and assignee not in df_dict['failed_by']:
                df_dict['failed_by'].add(assignee)
        df_dict['completed_by'] = list(df_dict['completed_by'])
        df_dict['failed_by'] = list(df_dict['failed_by'])
        df_list.append(df_dict)
df = pd.DataFrame(df_list).sort_values('closed_date')
df.tail(20)

,assignees,body,closed_date,comments,completed_by,created_date,failed_by,issue_num,labels,title
1926,[],Feedback from a recent workshop. Some particip...,2018-05-21 18:47:00,9,[devoncarew],2017-09-14 20:06:50,[sethladd],12095,[],Workshop attendees were curious why hot reload...
2245,[],"Below is a fairly simple code, however a rathe...",2018-05-21 20:11:04,4,[HansMuller],2018-05-18 01:48:10,[],17710,[],Error inside InputDecoration when using labelT...
2207,[],Something changed around SliverGrid. The regre...,2018-05-21 22:39:10,7,[],2018-04-18 01:49:55,[ng1905],16688,[],"GridView.{builder, custom} no longer work with..."
2013,[],devicelab test `flutter_view__start_up` timed ...,2018-05-21 22:43:35,1,[],2017-11-28 22:45:15,[aam],13233,[],devicelab flutter_view__start_up test timed out
1857,[],This is a problem for a11y. When an item is pa...,2018-05-21 23:33:48,1,[goderbauer],2017-08-17 20:19:37,[],11663,"[a: accessibility, f: scrolling]",showOnScreen / getOffsetToReveal doesn't take ...
2250,[],Using Flutter Gallery 0.4.4 (which now no long...,2018-05-22 14:46:58,0,[HansMuller],2018-05-21 18:41:24,[],17778,[team: gallery],Splash sticks on when attempting a fling scroll
2247,[HansMuller],I think there is an error with support for the...,2018-05-22 14:47:40,1,[HansMuller],2018-05-21 13:53:03,[],17770,"[a: internationalization, framework]",Bug in Norwegian language support.
2242,[Hixie],I have a driver test method that looks like th...,2018-05-22 18:55:47,7,[tvolkert],2018-05-15 04:25:04,[Hixie],17597,"[customer: mulligan (g3), t: flutter driver, t...",Failed assert in ScrollPosition.setPixels()
884,[],The documentation for assets should be clear a...,2018-05-22 20:16:00,3,[fmatosqg],2016-07-15 17:43:04,[],4927,[],Asset documentation should have a simple intro.
2246,[],![screen shot 2018-05-19 at 6 32 45 pm](https:...,2018-05-22 22:57:42,0,[HansMuller],2018-05-20 01:33:48,[],17750,"[a: tablet, team: gallery]",New Gallery doesn't look right on iPad


In [555]:
success_events = 0
partial_events = 0 # local commit / failed pr / assigned but didnt complete (make sure to filter completed events)

In [556]:
num_correct_k = 0
num_correct_total = 0
num_failed_k = 0
num_failed_total = 0
k = 1
dev_counts = {}
devs = set()
for _, row in df.iterrows():
    for failer in row['failed_by']:
        if dev_counts.get(failer, 0) == (k-1):
                num_failed_k += 1
        num_failed_total += 1
        devs.add(failer)
    for completer in row['completed_by']:
        if dev_counts.get(completer, 0) == (k-1):
                num_correct_k += 1
        num_correct_total += 1
        devs.add(completer)
        dev_counts[completer] = dev_counts.get(completer, 0) + 1
print("Num correct events for first " + str(k) + " contributor: " + str(num_correct_k)  )
print("Total events for first k contributor: " + str(num_correct_k + num_failed_k))
print()
print("Num correct events for all assignees: " + str(num_correct_total))
print("Total events for all assignees: " + str(num_correct_total+num_failed_total))
print()
print("Total number of unique devs: " + str(len(devs)))

Num correct events for first 1 contributor: 104
Total events for first k contributor: 263

Num correct events for all assignees: 2309
Total events for all assignees: 2843

Total number of unique devs: 148
